In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_purchases2020 = pd.read_csv("../data/raw/purchases_individuals_2020.csv")
df_purchases2019 = pd.read_csv("../data/raw/purchases_individuals_2019.csv")

In [ ]:
df_purchases = pd.concat([df_purchases2019, df_purchases2020], ignore_index=True) #concatenating both datasets

df_purchases

In [ ]:
def empty_cleaner(df):
    df_cleaned = df.dropna(axis=1, how='all') #removing all columns with all NaN values
    df_cleaned = df_cleaned.dropna(axis=1, how='all') #removing all columns with all NaN values
    return df_cleaned
    
df_purchases_cleaned = empty_cleaner(df_purchases) # apply the cleanning function

df_purchases_cleaned

In [ ]:
df_purchases_cleaned["OBS_FLAG"].value_counts() # checking values on OBS_FLAG

In [ ]:
df_purchases_cleaned_v2 = df_purchases_cleaned.drop(columns = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "OBS_FLAG"]) # dropping not required columns

df_purchases_cleaned_v2['indic_is'] = df_purchases_cleaned_v2['indic_is'].str.replace("Last online purchase: ", "", regex=False) # removing "Last online purchase: " from column indic_is

df_purchases_cleaned_v2

In [ ]:
df_purchases_cleaned_v3 = df_purchases_cleaned_v2.pivot(index=['indic_is', "ind_type", 'geo'], columns='TIME_PERIOD', values='OBS_VALUE').reset_index() #pivoting to add years on columns

df_purchases_cleaned_v3.columns.name = None # remove the name of the column index for better visualization

df_purchases_cleaned_v3.rename(columns={"geo": "country", "indic_is": "last_online_purchase", "ind_type": "grouped_individuals"}, inplace=True) # better names on columns :)

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_cleaned_v3 = df_purchases_cleaned_v3[~df_purchases_cleaned_v3['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]


df_purchases_cleaned_v3

In [ ]:
# in case we want to have the main data source filtered by European regions:

df_per_region = df_purchases_cleaned_v3.copy()

from dictionaries import regions # importing regions dictionary from dictionaries.py

# Create a reverse mapping from country to region
country_to_region = {country: region for region, countries in regions.items() for country in countries}

# Add a 'region' column to the new DataFrame
df_per_region['region'] = df_per_region['country'].map(country_to_region)


df_per_region

In [ ]:
# create a dummy function to send to the exploration notebook -Esteban <3
def import_df():
    return df_purchases_cleaned_v3, df_per_region